# Transfer Learning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import cv2
import os
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential

In [3]:
# ONLY FOR NVIDIA
#CUDA_VISIBLE_DEVICES = 0

In [5]:
#tf.config.list_physical_devices('GPU')

In [8]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, cache_dir='.', untar=True)

In [17]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [19]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [21]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [23]:
flowers_images_dict['roses'][:5]

[PosixPath('datasets/flower_photos/roses/10503217854_e66a804309.jpg'),
 PosixPath('datasets/flower_photos/roses/19823402005_2db025dd66_m.jpg'),
 PosixPath('datasets/flower_photos/roses/12240577184_b0de0e53ea_n.jpg'),
 PosixPath('datasets/flower_photos/roses/563847503_89e9756c80.jpg'),
 PosixPath('datasets/flower_photos/roses/3829990289_c0c3821e4d_m.jpg')]

In [25]:
def resize_images(df):
    X, y = [], []
    for flower_name, images in flowers_images_dict.items():
        for image in images:
            img = cv2.imread(str(image))
            resized_img = cv2.resize(img, (224,224))
            X.append(resized_img)
            y.append(flowers_labels_dict[flower_name])
    return X, y

In [27]:
X, y = resize_images(flowers_images_dict)

In [28]:
X = np.array(X)
y = np.array(y)

# Scale/split Data

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [32]:
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

2752 918
2752 918


In [33]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [34]:
IMAGE_SHAPE = (224,224)
IMAGE_SHAPE+(3,)


(224, 224, 3)

# ANN Model

In [36]:
from sklearn.metrics import classification_report, confusion_matrix

In [78]:
ann = Sequential([
    Flatten(),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(10, activation='softmax')
])
ann.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [80]:
ann.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

I0000 00:00:1724410666.348442    5905 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724410666.351520    5905 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724410666.354395    5905 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724410666.470425    5905 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [82]:
ann.fit(X_train, y_train, epochs=5)

2024-08-23 03:57:53.919795: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1657012224 exceeds 10% of free system memory.
2024-08-23 03:57:55.512674: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1657012224 exceeds 10% of free system memory.


Epoch 1/5


I0000 00:00:1724410677.133390    6265 service.cc:146] XLA service 0x7c4f20005830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724410677.133436    6265 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-08-23 03:57:57.161526: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-23 03:57:57.271904: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


19/86 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1409 - loss: 38.1797  

I0000 00:00:1724410678.548696    6265 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.2287 - loss: 22.4581
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4006 - loss: 2.5762
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4114 - loss: 2.8988
Epoch 4/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4961 - loss: 1.8592
Epoch 5/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4899 - loss: 2.0394


In [84]:
y_pred = ann.predict(X_test)
y_pred_classes = [np.argmax(element) for element in y_pred]
print(classification_report(y_test, y_pred_classes))

2024-08-23 03:58:07.124793: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 552738816 exceeds 10% of free system memory.
2024-08-23 03:58:07.649053: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 552738816 exceeds 10% of free system memory.


29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
              precision    recall  f1-score   support

           0       0.35      0.68      0.46       176
           1       0.27      0.54      0.36       154
           2       0.51      0.28      0.36       226
           3       0.63      0.38      0.47       150
           4       0.42      0.12      0.18       212
           7       0.00      0.00      0.00         0

    accuracy                           0.38       918
   macro avg       0.37      0.33      0.31       918
weighted avg       0.44      0.38      0.36       918



/home/shawn/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/shawn/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/shawn/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Transfer Learning

In [47]:
#print(hub.version)

In [45]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
import tf_keras

In [47]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224,224,3), trainable=False, 
)

I0000 00:00:1724411090.332206    6547 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724411090.335436    6547 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724411090.338149    6547 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724411090.455840    6547 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [48]:
num_of_flowers = 5

tf_keras

model = tf_keras.Sequential([
    pretrained_model_without_top_layer,
    tf_keras.layers.Dense(num_of_flowers, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2264389 (8.64 MB)
Trainable params: 6405 (25.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [49]:
model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [53]:
model.fit(X_train, y_train, epochs=20)

2024-08-23 04:04:56.043853: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1657012224 exceeds 10% of free system memory.
2024-08-23 04:04:57.827084: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1657012224 exceeds 10% of free system memory.


Epoch 1/20


2024-08-23 04:05:00.739886: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1724411100.885419    7195 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411100.939058    7195 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411100.940286    7195 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411100.950434    7195 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411100.952981    7195 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411100.955331    7195 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411100.957817    7195 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411100.959497    7195 gpu_t

86/86 [==============================] - 5s 27ms/step - loss: 0.8318 - accuracy: 0.6955
Epoch 2/20
86/86 [==============================] - 2s 27ms/step - loss: 0.4142 - accuracy: 0.8608
Epoch 3/20
86/86 [==============================] - 2s 27ms/step - loss: 0.3272 - accuracy: 0.8943
Epoch 4/20
86/86 [==============================] - 2s 27ms/step - loss: 0.2736 - accuracy: 0.9106
Epoch 5/20
86/86 [==============================] - 2s 27ms/step - loss: 0.2340 - accuracy: 0.9328
Epoch 6/20
86/86 [==============================] - 2s 28ms/step - loss: 0.2060 - accuracy: 0.9422
Epoch 7/20
86/86 [==============================] - 2s 27ms/step - loss: 0.1820 - accuracy: 0.9542
Epoch 8/20
86/86 [==============================] - 2s 27ms/step - loss: 0.1681 - accuracy: 0.9604
Epoch 9/20
86/86 [==============================] - 2s 28ms/step - loss: 0.1486 - accuracy: 0.9655
Epoch 10/20
86/86 [==============================] - 2s 28ms/step - loss: 0.1341 - accuracy: 0.9698
Epoch 11/20
86/86 [=

In [54]:
model.evaluate(X_test, y_test)

29/29 [==============================] - 1s 30ms/step - loss: 0.3375 - accuracy: 0.8922


W0000 00:00:1724411150.962152    7193 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411150.963093    7193 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411150.964040    7193 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411150.966593    7193 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411150.967853    7193 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411150.969444    7193 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411150.970676    7193 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411150.971651    7193 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724411150.972624    7193 gp

[0.3375411629676819, 0.8921568393707275]